In [20]:
import cv2
import numpy as np
from statistics import mean, median

In [21]:
def calculateSpatialInfo(img: np.ndarray) -> float:
    """
    Calculate spatial information from a grayscale image using Sobel filters.

    Args:
        img: A grayscale image as a numpy array.

    Returns:
        The spatial information as a float.
    """
    sh = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=1)
    sv = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=1)

    sobel_image = np.sqrt(np.square(sh) + np.square(sv))

    si_mean = np.sum(sobel_image) / (sobel_image.shape[0] * sobel_image.shape[1])
    si_rms = np.sqrt(np.sum(sobel_image ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))
    si_stdev = np.sqrt(np.sum(sobel_image ** 2 - si_mean ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))

    return si_stdev

In [22]:
a_gray = cv2.imread('images/a.jpg',cv2.IMREAD_GRAYSCALE)
b_gray = cv2.imread('images/b.jpg',cv2.IMREAD_GRAYSCALE)
c_gray = cv2.imread('images/c.jpg',cv2.IMREAD_GRAYSCALE)
print(f'Spatial Info of A: {calculateSpatialInfo(a_gray)}')
print(f'Spatial Info of B: {calculateSpatialInfo(b_gray)}')
print(f'Spatial Info of C: {calculateSpatialInfo(c_gray)}')

Spatial Info of A: 20.65579411743317
Spatial Info of B: 23.20730158646722
Spatial Info of C: 45.468247897791535


In [23]:
a = cv2.imread('images/a.jpg')
b = cv2.imread('images/b.jpg')
c = cv2.imread('images/c.jpg')

In [24]:
def resizeImage(original: np.ndarray, x_rate: float, y_rate: float) -> np.ndarray:
    shape = list(original.shape)
    shape[0] = int(shape[0]*x_rate)
    shape[1] = int(shape[1]*y_rate)
    resized = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            orig_i, orig_j = int(i/x_rate),int(j/y_rate)
            resized[i][j] = [*original[orig_i][orig_j]]
    return resized


In [25]:
a_resized = resizeImage(a, 2, 2)
cv2.imwrite('images/a_resized.jpg', a_resized)
b_resized = resizeImage(b, 2, 2)
cv2.imwrite('images/b_resized.jpg', b_resized)
c_resized = resizeImage(c, 2, 2)
cv2.imwrite('images/c_resized.jpg', c_resized)

True

In [26]:
def getNeighbour(image: np.ndarray, i: int, j: int) -> list:
    neighbour = [[], [], []]
    shape = image.shape
    for di, dj in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
        curr_i = i + di
        curr_j = j + dj
        if curr_i < 0 or curr_i >= shape[0]:
            continue
        if curr_j < 0 or curr_j >= shape[1]:
            continue
        for color in [0, 1, 2]:
            neighbour[color].append(image[curr_i][curr_j][color])
    return neighbour


In [27]:
def polishingImage(resized: np.ndarray, func: callable) -> np.ndarray:
    shape = resized.shape
    polished = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            neighbour = getNeighbour(resized, i, j)
            polished[i][j] = [func(neighbour[0]), func(neighbour[1]), func(neighbour[2])]
    return polished

In [28]:
a_polished = polishingImage(a_resized, median)
cv2.imwrite('images/a_polished.jpg', a_polished)
b_polished = polishingImage(b_resized, median)
cv2.imwrite('images/b_polished.jpg', b_polished)
c_polished = polishingImage(c_resized, median)
cv2.imwrite('images/c_polished.jpg', c_polished)